# CORD Dataset Quick Exploration

In [4]:
# !pip install easyimages

In [5]:
!which python

/home/vantuan5644/anaconda3/envs/ReceiptOCR/bin/python


In [6]:
import sys
sys.executable

'/home/vantuan5644/anaconda3/envs/ReceiptOCR/bin/python'

In [7]:
from easyimages import EasyImageList

In [8]:
image_list = EasyImageList.from_folder('../datasets/CORD/CORD/train/image')

In [9]:
image_list.symlink_images()

In [10]:
%matplotlib inline

In [11]:
image_list.html(sample=10, size=20)

## Padding images

In [12]:
import sys
from random import randint
import cv2 as cv


In [16]:
import os


In [20]:
path = '../datasets/COOP/train/img_1.jpg'
os.path.split(path)[1]

'img_1.jpg'

In [27]:
def padding_image(img_path, output_dir=None):
    src = cv.imread(img_path, cv.IMREAD_COLOR)

    borderType = cv.BORDER_REPLICATE

    top = int(0.05 * src.shape[0])  # shape[0] = rows
    bottom = top
    left = int(0.05 * src.shape[1])  # shape[1] = cols
    right = left

    dst = cv.copyMakeBorder(src, top, bottom, left, right, borderType, None, value)
    
    file_name = os.path.splitext(os.path.split(path)[1]) # tuple
    if output_dir is None:
        output_dir = os.path.split(img_path)[0]
    new_file = os.path.join(output_dir, file_name[0] + '_padded' + file_name[1])
    cv.imwrite(new_file, dst)

In [30]:
dataset_dir = '../datasets/COOP/train'
output_dir = '../datasets/COOP/padded'

for img in os.listdir(dataset_dir):
    img_path = os.path.join(dataset_dir, img)
    padding_image(img_path, output_dir = output_dir)